In [1]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.layers import (Conv2D, UpSampling2D, Input, MaxPool2D,
                                    Activation, Flatten, Dropout, BatchNormalization,
                                    Concatenate)
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

In [3]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/kvasir-seg.zip", 'r')
zip_ref.extractall("/content/kvasir-seg")
zip_ref.close()

In [4]:
H, W = 256, 256

In [5]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)

    intersection = tf.reduce_sum(y_true * y_pred)

    return (2 * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

In [6]:
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [7]:
def conv_block(x, num_filters, act=True) :
    x = Conv2D(num_filters, kernel_size=3, padding="same")(x)

    if act==True:
        x = BatchNormalization()(x)
        x = Activation("relu")(x)

    return x

In [8]:
def encoder_block(x, num_filters):
    x = conv_block(x, num_filters)
    x = conv_block(x, num_filters)

    p = MaxPool2D((2, 2))(x)

    return x, p

In [9]:
def unet3plus(input_shape, num_classes=1):

    """Inputs"""
    inputs = Input(input_shape, name="input_layer")

    """Pre_trained VGG16 Model"""
    encoder = ResNet50(include_top=False, weights="imagenet", input_tensor=inputs)

    """Encoder"""
    e1 = encoder.get_layer("input_layer").output
    e2 = encoder.get_layer("conv1_relu").output
    e3 = encoder.get_layer("conv2_block3_out").output
    e4 = encoder.get_layer("conv3_block4_out").output

    """Bridge"""
    e5 = encoder.get_layer("conv4_block6_out").output

    """Decoder 4"""
    e1_d4 = MaxPool2D((8, 8))(e1)
    e1_d4 = conv_block(e1_d4, 64)

    e2_d4 = MaxPool2D((4, 4))(e2)
    e2_d4 = conv_block(e2_d4, 64)

    e3_d4 = MaxPool2D((2, 2))(e3)
    e3_d4 = conv_block(e3_d4, 64)

    e4_d4 = conv_block(e4, 64)

    e5_d4 = UpSampling2D((2, 2), interpolation="bilinear")(e5)
    e5_d4 = conv_block(e5_d4, 64)

    d4 = Concatenate()([e1_d4, e2_d4, e3_d4, e4_d4, e5_d4])
    d4 = conv_block(d4, 64*5)

    """Decoder 3"""
    e1_d3 = MaxPool2D((4, 4))(e1)
    e1_d3 = conv_block(e1_d3, 64)

    e2_d3 = MaxPool2D((2, 2))(e2)
    e2_d3 = conv_block(e2_d3, 64)

    #e3_d3 = MaxPool2D((2, 2))(e3)
    e3_d3 = conv_block(e3, 64)

    e4_d3 = UpSampling2D((2, 2), interpolation="bilinear")(d4)
    e4_d3 = conv_block(e4_d3, 64)

    e5_d3 = UpSampling2D((4, 4), interpolation="bilinear")(e5)
    e5_d3 = conv_block(e5_d3, 64)

    d3 = Concatenate()([e1_d3, e2_d3, e3_d3, e4_d3, e5_d3])
    d3 = conv_block(d3, 64*5)

    """Decoder 2"""
    e1_d2 = MaxPool2D((2, 2))(e1)
    e1_d2 = conv_block(e1_d2, 64)

    #e2_d2 = MaxPool2D((2, 2))(e2)
    e2_d2 = conv_block(e2, 64)

    e3_d2 = UpSampling2D((2, 2), interpolation="bilinear")(d3)
    e3_d2 = conv_block(e3_d2, 64)

    e4_d2 = UpSampling2D((4, 4), interpolation="bilinear")(d4)
    e4_d2 = conv_block(e4_d2, 64)

    e5_d2 = UpSampling2D((8, 8), interpolation="bilinear")(e5)
    e5_d2 = conv_block(e5_d2, 64)

    d2 = Concatenate()([e1_d2, e2_d2, e3_d2, e4_d2, e5_d2])
    d2 = conv_block(d2, 64*5)

    """Decoder 1"""
    #e1_d1 = MaxPool2D((2, 2))(e1)
    e1_d1 = conv_block(e1, 64)

    d2_d1 = UpSampling2D((2, 2), interpolation="bilinear")(d2)
    d2_d1 = conv_block(d2_d1, 64)

    d3_d1 = UpSampling2D((4, 4), interpolation="bilinear")(d3)
    d3_d1 = conv_block(d3_d1, 64)

    d4_d1 = UpSampling2D((8, 8), interpolation="bilinear")(d4)
    d4_d1 = conv_block(d4_d1, 64)

    e5_d1 = UpSampling2D((16, 16), interpolation="bilinear")(e5)
    e5_d1 = conv_block(e5_d1, 64)

    d1 = Concatenate()([e1_d1, d2_d1, d3_d1, d4_d1, e5_d1])
    d1 = conv_block(d1, 64*5)

    """Output"""
    y1 = Conv2D(num_classes, kernel_size=3, padding="same")(d1)
    y1 = Activation("sigmoid")(y1)
    outputs = [y1]

    model = tf.keras.Model(inputs, outputs)
    return model

In [10]:
if __name__ == "__main__":
    input_shape = (256, 256, 3)
    model = unet3plus(input_shape)
    model.summary()

94765736/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_layer (InputLayer)    [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_layer[0][0]']         
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1_conv[0][0]']          
 on)                         

In [10]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [11]:
def load_dataset(path, split=0.1):
    """ Loading the images and masks """
    X = sorted(glob(os.path.join(path, "images", "*.jpg")))
    y = sorted(glob(os.path.join(path, "masks", "*.jpg")))

    """ Spliting the data into training and testing """
    split_size = int(len(X) * split)

    X_train, X_val = train_test_split(X, test_size=split_size, random_state=42)
    y_train, y_val = train_test_split(y, test_size=split_size, random_state=42)

    X_train, X_test = train_test_split(X_train, test_size=split_size, random_state=42)
    y_train, y_test = train_test_split(y_train, test_size=split_size, random_state=42)

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [12]:
def read_image(path):
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(image, (H, W))
    img = img / 255.0
    img = img.astype(np.float32)
    return img

In [13]:
def read_mask(path):
    path = path.decode()
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (H, W))
    mask = mask / 255.0
    mask = mask.astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    return mask

In [14]:
def tf_parse(x, y):
    def parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [15]:
def tf_dataset(x, y, batch=2):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(tf_parse).batch(batch).prefetch(10)
    return ds

In [25]:
from warnings import filters
data_path = "/content/kvasir-seg/Kvasir-SEG"
files = "/content/drive/MyDrive/kvasir-seg/Kvasir-SEG"
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for sorting files """
    create_dir(files + "/files")

    """ Hyperparameters """
    batch_size = 2
    lr = 1e-5
    num_epochs = 20
    model_path = os.path.join(files, "files", "Polyp Segmentation using Unet3_30_epochs).h5")
    csv_path = os.path.join(files, "files", "Polyp Segmentation using Unet3_30_epochs.csv")

    """Dataset"""
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_dataset(data_path)

    print(f"Training: /t{len(X_train)} - {len(y_train)}")
    print(f"Validataion: /t{len(X_val)} - {len(y_val)}")
    print(f"testing: /t{len(X_test)} - {len(y_test)}")

    train_dataset = tf_dataset(X_train, y_train, batch=batch_size)
    val_dataset = tf_dataset(X_val, y_val, batch=batch_size)

    #model = unet3plus((H, W, 3))
    model = load_model("/content/drive/MyDrive/kvasir-seg/Kvasir-SEG/files/Polyp Segmentation using Unet3.h5", compile=False)
    model.compile(loss=dice_loss, optimizer=Adam(learning_rate=lr), metrics=[dice_coef])
    #model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-10, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor="val_loss", patience=50, restore_best_weights=False)
    ]

Training: /t800 - 800
Validataion: /t100 - 100
testing: /t100 - 100


In [26]:
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=val_dataset,
    callbacks=callbacks
)

Epoch 1/20
400/400 [==============================] - ETA: 0s - loss: 0.0252 - dice_coef: 0.9748
Epoch 1: val_loss improved from inf to 0.13489, saving model to /content/drive/MyDrive/kvasir-seg/Kvasir-SEG/files/Polyp Segmentation using Unet3_30_epochs).h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


400/400 [==============================] - 242s 504ms/step - loss: 0.0252 - dice_coef: 0.9748 - val_loss: 0.1349 - val_dice_coef: 0.8651 - lr: 1.0000e-05
Epoch 2/20
400/400 [==============================] - ETA: 0s - loss: 0.0229 - dice_coef: 0.9771
Epoch 2: val_loss did not improve from 0.13489
400/400 [==============================] - 201s 503ms/step - loss: 0.0229 - dice_coef: 0.9771 - val_loss: 0.1423 - val_dice_coef: 0.8577 - lr: 1.0000e-05
Epoch 3/20
400/400 [==============================] - ETA: 0s - loss: 0.0214 - dice_coef: 0.9786
Epoch 3: val_loss did not improve from 0.13489
400/400 [==============================] - 203s 506ms/step - loss: 0.0214 - dice_coef: 0.9786 - val_loss: 0.1433 - val_dice_coef: 0.8567 - lr: 1.0000e-05
Epoch 4/20
400/400 [==============================] - ETA: 0s - loss: 0.0205 - dice_coef: 0.9795
Epoch 4: val_loss did not improve from 0.13489
400/400 [==============================] - 195s 487ms/step - loss: 0.0205 - dice_coef: 0.9795 - val_loss: 

KeyboardInterrupt: 

In [33]:
""" Testing Part """

if __name__ == "__main__":
    """Seeding"""
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for saving files """
    create_dir(files + f"/results")

    """ Load the model """
    #model_path = os.path.join(files, "files", "Polyp Segmentation using Unet3_30_epochs.h5")
    model_path = "/content/drive/MyDrive/kvasir-seg/Kvasir-SEG/files/Polyp Segmentation using Unet3_30_epochs.h5"
    model = load_model(model_path, compile=False)

    """ Prediction """
    for x, y in tqdm(zip(X_test, y_test), total=len(X_test)):
        """ Extracting the name """
        #print("X: ", x, "y: ", y)
        name = x.split("/")[-1].split(".")[0]
        print(name)

        """ Reading the image """
        img = cv2.imread(x, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (H, W))
        x = img / 255.0
        x = np.expand_dims(x, axis=0)

        """ Reading the mask"""
        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (H, W))
        mask = mask / 255.0
        mask = np.expand_dims(mask, axis=-1)
        mask = np.concatenate([mask, mask, mask], axis=-1)

        """ Predicton """
        pred = model.predict(x, verbose=0)[0]
        pred = np.concatenate([pred, pred, pred], axis=-1)
        #pred = (pred > 0.5).astype(np.int32)

        """ Save final mask """
        line = np.ones((H, 10, 3)) * 255
        cat_images = np.concatenate([img, line, mask*255, line, pred*255], axis=1)
        save_image_path = os.path.join(files, "results", f"{name}.jpg")
        cv2.imwrite(save_image_path, cat_images)

  0%|          | 0/100 [00:00<?, ?it/s]

cju33za6l2qy70988jhrlp2ev


  2%|▏         | 2/100 [00:01<01:09,  1.42it/s]

cju1ejj7dvqfa0835ra184v5m
cju30gxjq0djk0988jytm49rs


  4%|▍         | 4/100 [00:01<00:33,  2.89it/s]

...

cju2wxv0hxs2f09884w48v8fi
cju8bm24yrrdp081829mbo8ic


 98%|█████████▊| 98/100 [00:18<00:00,  4.98it/s]

cju3x2s11ibzi0817kk284k0j


100%|██████████| 100/100 [00:19<00:00,  5.18it/s]

cju7ev2b12owa08500bpfpwyw
